# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, ml_methods_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils
from neural_data_analysis.design_kits.design_by_segment import create_design_df, predictor_utils, other_feats
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event, pn_glm_utils
from neural_data_analysis.neural_analysis_tools.glm_tools.tpg import glm_bases, glm_plotting, glm_plotting2, glm_fit

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_psth import core_stops_psth, psth_postprocessing, psth_stats, compare_events, dpca_utils
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_fit import stop_glm_fit, cv_stop_glm, glm_fit_utils, variance_explained
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_plotting import plot_spikes, plot_glm_fit, plot_tuning_func, compare_glm_fit
from neural_data_analysis.design_kits.design_around_event import event_binning, stop_design, cluster_design, design_checks
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_hyperparams import compare_glm_configs, glm_hyperparams_class
from neural_data_analysis.design_kits.design_by_segment import spike_history
from neural_data_analysis.topic_based_neural_analysis.full_session import create_full_session_design
from neural_data_analysis.design_kits.design_by_segment import temporal_feats

import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload
import json

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
from numpy import pi

from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve
# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

# Retrieve data

## get data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0416"

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0312"

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"

In [ ]:
reload(pn_aligned_by_event)
reload(pn_utils)

In [ ]:
reduce_y_var_lags = False
planning_data_by_point_exists_ok = True
y_data_exists_ok = True

pn = pn_aligned_by_event.PlanningAndNeuralEventAligned(raw_data_folder_path=raw_data_folder_path)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=planning_data_by_point_exists_ok)
#pn.get_x_and_y_data_for_modeling(exists_ok=y_data_exists_ok, reduce_y_var_lags=reduce_y_var_lags)

pn.rebin_data_in_new_segments(cur_or_nxt='cur', first_or_last='first', time_limit_to_count_sighting=2,
                                 start_t_rel_event=0, end_t_rel_event=1.5, rebinned_max_x_lag_number=2)

for col in ['cur_vis', 'nxt_vis', 'cur_in_memory', 'nxt_in_memory']:
    pn.rebinned_y_var[col] = (pn.rebinned_y_var[col] > 0).astype(int)
    
rebinned_x_var = pn.rebinned_x_var.copy()
rebinned_y_var = pn.rebinned_y_var.copy()

rebinned_y_var.shape


## select subset

### get heading data

In [ ]:
heading_info_df, heading_df = pn_glm_utils.get_test_heading_df(raw_data_folder_path)

In [ ]:
# rebinned_x_var, rebinned_y_var = pn_glm_utils.select_ff_subset(heading_df, pn.rebinned_x_var, pn.rebinned_y_var, 
#                                                                top=False, pct=0.5)
# rebinned_x_var = pn_glm_utils.drop_constant_columns(rebinned_x_var)

# GLM data

In [ ]:
reload(create_design_df)
reload(other_feats)

data = rebinned_y_var.copy()
trial_ids = data['new_segment']
dt = pn.bin_width

design_df, meta0, meta = create_design_df.get_initial_design_df(data, dt, trial_ids)

cluster_num = 3
y = rebinned_x_var[f'cluster_{cluster_num}']

chk = predictor_utils.check_design_vs_bases(design_df, meta, strict=True)
assert chk['ok'], chk['problems']


cluster_cols = [col for col in rebinned_x_var.columns if col.startswith('cluster_')]
df_Y = rebinned_x_var[cluster_cols]
df_Y.columns = df_Y.columns.str.replace('cluster_', '').astype(int)



# Exp

In [ ]:
pn.planning_data_by_bin.shape

In [ ]:
pn.planning_data_by_point.shape

In [ ]:
general_utils.check_na_in_df(pn.planning_data_by_point)

In [ ]:
pn.prep_data_to_analyze_planning()

In [ ]:
pn.ctrl_inst.prep_behav_data_to_analyze_planning(planning_data_by_point_exists_ok=False)

In [ ]:
pn.planning_data_by_point

In [ ]:
pn.planning_data_by_point['point_index'].diff().max()

In [ ]:
pn.planning_data_by_point.columns.tolist()

In [ ]:
pn.ctrl_inst.both_ff_across_time_df

In [ ]:
pn.bin_info

In [ ]:
new_seg_info = pd.DataFrame({
    'new_segment': 0,
    'new_seg_start_time': max(0, pn.ff_caught_T_sorted.min() - 1),
    'new_seg_end_time': pn.ff_caught_T_sorted.max(),
    'new_seg_duration': pn.ff_caught_T_sorted.max() - max(0, pn.ff_caught_T_sorted.min() - 1)
}, index=[0])

In [ ]:
reload(pn_utils)

# rebin monkey info

In [ ]:
# add num_ff_visible etc.
pn.make_or_retrieve_ff_dataframe()
pn.monkey_information = pn_utils.add_ff_visible_or_in_memory_info_by_point(
    pn.monkey_information, pn.ff_dataframe)


In [ ]:
rebinned_monkey_data = pn_utils.rebin_all_segments_local_bins(
            pn.monkey_information, new_seg_info, bin_width=pn.bin_width, respect_old_segment=False,
            add_bin_edges=True,
            )

rebinned_monkey_data = temporal_feats.add_stop_and_capture_columns(rebinned_monkey_data, trial_ids, pn.ff_caught_T_new)

dt = pn.bin_width
trial_ids = np.repeat(0, len(rebinned_monkey_data))
design_df, meta0, meta = create_full_session_design.get_initial_full_session_design_df(rebinned_monkey_data, dt, trial_ids)


In [ ]:
design_df

# relevant pn features

In [ ]:
from neural_data_analysis.topic_based_neural_analysis.full_session import selected_features

In [ ]:
# also need curv_of_traj

In [ ]:
selected_pn_features = (
    selected_features.cur_and_nxt_ff_visibility
    + selected_features.firefly_geometry
    + selected_features.firefly_geometry_abs
    + selected_features.firefly_relations
    + selected_features.firefly_relations_abs
    + selected_features.angle_differences
    + selected_features.curvature
    + selected_features.curvature_optimal
    + selected_features.curvature_residuals
    + selected_features.curvature_summary
    + selected_features.eye_raw
    + selected_features.stop_clusters
)


In [ ]:
selected_kinematics_features = (
    selected_features.global_time
    + selected_features.core_kinematics
    + selected_features.motion_signals
    + selected_features.gaze
    + selected_features.derived_motion
)


In [ ]:
rebinned_monkey_data['stop'] = (rebinned_monkey_data['stop_id'] > 0)

In [ ]:
rebinned_monkey_data[selected_kinematics_features]

In [ ]:
bins_2d = rebinned_monkey_data[['bin_left', 'bin_right']].values
out = pn_utils.rebin_all_segments_global_bins(
    pn.planning_data_by_point,
    pn.new_seg_info,
    bins_2d=bins_2d,
    how='mean',
    respect_old_segment=True,
    require_full_bin=True,
    add_bin_edges=True,
    add_support_duration=True,
)

In [ ]:
out[selected_pn_features]

then, I need to add features from glm_around_stops

https://chatgpt.com/c/6964877c-61a0-832e-9baa-41007244b4b4


In [ ]:
don't forget const....